In [1]:
training_file = 'warpeace_input.txt'
raw_text = open(training_file, 'r').read()
raw_text = raw_text.lower()
raw_text[:100]

'ï»¿"well, prince, so genoa and lucca are now just family estates of the\nbuonapartes. but i warn you,'

In [2]:
n_chars = len(raw_text)
print('Total characters: {}'.format(n_chars))

Total characters: 3196232


In [3]:
chars = sorted(list(set(raw_text)))
n_vocab = len(chars)
print('Total vocabulary (unique characters): {}'.format(n_vocab))
print(chars)

Total vocabulary (unique characters): 60
['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '¤', '©', 'ª', '»', '¿', 'ã', 'ï']


In [4]:
index_to_char = dict((i, c) for i, c in enumerate(chars))
char_to_index = dict((c, i) for i, c in enumerate(chars))
print(char_to_index)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, '*': 7, ',': 8, '-': 9, '.': 10, '/': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, ';': 23, '=': 24, '?': 25, 'a': 26, 'b': 27, 'c': 28, 'd': 29, 'e': 30, 'f': 31, 'g': 32, 'h': 33, 'i': 34, 'j': 35, 'k': 36, 'l': 37, 'm': 38, 'n': 39, 'o': 40, 'p': 41, 'q': 42, 'r': 43, 's': 44, 't': 45, 'u': 46, 'v': 47, 'w': 48, 'x': 49, 'y': 50, 'z': 51, '\xa0': 52, '¤': 53, '©': 54, 'ª': 55, '»': 56, '¿': 57, 'ã': 58, 'ï': 59}


In [5]:
import numpy as np
seq_length = 190
n_seq = int(n_chars / seq_length)

In [6]:
X = np.zeros((n_seq, seq_length, n_vocab))
Y = np.zeros((n_seq, seq_length, n_vocab))

In [7]:
for i in range(n_seq):
    x_sequence = raw_text[i * seq_length : (i + 1) * seq_length]
    x_sequence_ohe = np.zeros((seq_length, n_vocab))
    for j in range(seq_length):
        char = x_sequence[j]
        index = char_to_index[char]
        x_sequence_ohe[j][index] = 1.
    X[i] = x_sequence_ohe
    y_sequence = raw_text[i * seq_length + 1 : (i + 1) * seq_length + 1]
    y_sequence_ohe = np.zeros((seq_length, n_vocab))
    for j in range(seq_length):
        char = y_sequence[j]
        index = char_to_index[char]
        y_sequence_ohe[j][index] = 1.
    Y[i] = y_sequence_ohe

In [8]:
X.shape

(16822, 190, 60)

In [9]:
Y.shape

(16822, 190, 60)

In [12]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import SimpleRNN
from keras.layers import TimeDistributed
from keras import optimizers
from tensorflow import keras

In [13]:
batch_size = 100
n_layer = 2
hidden_units = 800
n_epoch = 300
dropout = 0.3

In [15]:
model = Sequential()

model.add(SimpleRNN(hidden_units, activation='relu', input_shape=(None, n_vocab), return_sequences=True))
model.add(Dropout(dropout))

for i in range(n_layer - 1):
    model.add(SimpleRNN(hidden_units, activation='relu', return_sequences=True))
    model.add(Dropout(dropout))
    
model.add(TimeDistributed(Dense(n_vocab)))
model.add(Activation('softmax'))

In [22]:
optimizer = keras.optimizers.legacy.RMSprop(learning_rate=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

In [24]:
model.compile(loss= "categorical_crossentropy", optimizer=optimizer)

In [26]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 simple_rnn_2 (SimpleRNN)    (None, None, 800)         688800    
                                                                 
 dropout_2 (Dropout)         (None, None, 800)         0         
                                                                 
 simple_rnn_3 (SimpleRNN)    (None, None, 800)         1280800   
                                                                 
 dropout_3 (Dropout)         (None, None, 800)         0         
                                                                 
 time_distributed_1 (TimeDis  (None, None, 60)         48060     
 tributed)                                                       
                                                                 
 activation (Activation)     (None, None, 60)          0         
                                                      

In [27]:
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
filepath="weights/weights_layer_%d_hidden_%d_epoch_{epoch:03d}_loss_{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')

In [29]:
early_stop = EarlyStopping(monitor='loss', min_delta=0, patience=50, verbose=1, mode='min')

In [30]:
def generate_text(model, gen_length, n_vocab, index_to_char):
    """
    Generating text using the RNN model
    @param model: current RNN model
    @param gen_length: number of characters we want to generate
    @param n_vocab: number of unique characters
    @param index_to_char: index to character mapping
    @return:
    """
    # Start with a randomly picked character
    index = np.random.randint(n_vocab)
    y_char = [index_to_char[index]]
    X = np.zeros((1, gen_length, n_vocab))
    for i in range(gen_length):
        X[0, i, index] = 1.
        indices = np.argmax(model.predict(X[:, max(0, i - 99):i + 1, :])[0], 1)
        index = indices[-l]
        y_char.append(index_to_char[index])
    return ('').join(y_char)

In [32]:
class ResultChecker(Callback):
    def __init__(self, model, N, gen_length):
        self.model = model
        self.N = N
        self.gen_length = gen_length
        
    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.N == 0:
            result = generate_text(self.model, self.gen_length, n_vocab, index_to_char)
            print('\nMy War and Peace \n' + result)

In [35]:
model.fit(X, Y, batch_size=batch_size, verbose=1, epochs=n_epoch, 
          callbacks=[ResultChecker(model, 10, 200), checkpoint, early_stop])

Epoch 1/300
  2/169 [..............................] - ETA: 11:35 - loss: 8217718803012897472512.0000

KeyboardInterrupt: 